In [34]:
import keras.backend as K


from keras.applications.xception import Xception
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.layers import Concatenate, Lambda, Flatten, Dense, AveragePooling2D, Dropout, GlobalAveragePooling2D
from keras.metrics import top_k_categorical_accuracy
from keras.models import load_model, Model, save_model
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator

import math
import numpy as np
import os
import pickle
import sys
import tensorflow as tf
import pandas as pd

In [39]:
num_gpus_to_use = 1

model_name = "xception_v2"
models_folder  = "models"

img_width = 180
img_height = 180

batch_size = 8  # 258
num_epochs = [1, 1]
learning_rate = [0.1, 0.1, 0.1]
sub_model_names = ["trainhead_test", "blocks_11_test", "whole_model_test"]
train_data_dir = 'output/train'
val_data_dir = 'output/validation'

classnames = pickle.load(open("class_order.pkl", "rb"))
# classnames = pd.read_csv("input/category_names.csv", "rb")

1000012768


list

In [37]:
model0 = Xception(include_top=False, weights='imagenet',
                    input_tensor=None, input_shape=(img_width, img_height, 3))

for lay in model0.layers:
    lay.trainable = False

x = model0.output
x = GlobalAveragePooling2D(name='avg_pool')(x)

x = Dropout(0.2)(x)
x = Dense(len(classnames), activation='softmax', name='predictions')(x)
model = Model(model0.input, x)

## Generator

In [16]:
# Data generator
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size = (img_width, img_height),
        batch_size = batch_size,
        shuffle = True,
        classes = classnames,
        class_mode = 'categorical')

val_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle = True,
        classes = classnames,
        class_mode = 'categorical')

os.makedirs("./models", exist_ok=True)

Found 9916714 images belonging to 5270 classes.
Found 2454579 images belonging to 5270 classes.


## TRAIN

In [18]:
sub_model_save_paths = []
callbacks = []
had_been_run = []

for idx,subm in enumerate(sub_model_names):
    sub_model_save_paths.append(models_folder + "mod_and_outp_" + model_name + "_" + subm + ".hdf5")
    callbacks.append([ModelCheckpoint(monitor='val_loss',
                             filepath= models_folder + model_name + "_" + subm + '_{epoch:03d}-{val_loss:.7f}.hdf5',
                             save_best_only=False,
                             save_weights_only=False,
                             mode='max'),
                             TensorBoard(log_dir='logs/{}'.format(model_name))])

    if os.path.isfile(sub_model_save_paths[idx]):
        had_been_run.append(True)
    else:
        had_been_run.append(False)

## train HEAD

In [23]:
setattr(tf.contrib.rnn.GRUCell, '__deepcopy__', lambda self, _: self)
setattr(tf.contrib.rnn.BasicLSTMCell, '__deepcopy__', lambda self, _: self)
setattr(tf.contrib.rnn.MultiRNNCell, '__deepcopy__', lambda self, _: self)

In [24]:
#sub_model_names  = ["trainhead", "blocks11", "whole_model"]

sub_model_order = 0  # Train head (base 0)

if had_been_run[sub_model_order]:
    print("\n We FOUND",sub_model_save_paths[sub_model_order],".\n\t Since it seems to have been already launched, we will skip this model.\n\n")

else:

    print("\n Starting optimization of model:",sub_model_names[sub_model_order])

    model.compile(loss='categorical_crossentropy',
                    optimizer=SGD(lr=learning_rate[sub_model_order], momentum=0.9),
                    metrics=[top_k_categorical_accuracy, 'accuracy'])

    model.fit_generator(generator=train_generator,
                        steps_per_epoch=2000000,
                        verbose=0,
                        callbacks=callbacks[sub_model_order],
                        validation_data=validation_generator,
                        initial_epoch=0,
                        epochs = num_epochs[sub_model_order],
                        use_multiprocessing=True,
                        max_queue_size=10,
                        workers = 8,
                        validation_steps=10000)

    model.save(sub_model_save_paths[sub_model_order])


 Starting optimization of model: trainhead_test


StopIteration: can't pickle _thread.lock objects

## train BLOCKS 11+ 

In [ ]:
sub_model_order = 1  # blocks 11+ (base 0)

if had_been_run[sub_model_order]:
    print("\n We FOUND",sub_model_save_paths[sub_model_order],".\n\t We will skip this model.\n\n")

else:

    if had_been_run[sub_model_order - 1]:
        model = load_model(sub_model_save_paths[sub_model_order - 1])  # returns a compiled model identical to the previous one

    print("\n Starting optimization of model:",sub_model_names[sub_model_order])
    print('\n\n\n')
    print('Debugging Blocks11 layer issue:')
    for lay in model.layers:
        print(lay)

    print('\n\n\n')
    for clayer in model.layers[num_gpus_to_use + 1].layers:
        print("trainable:", clayer.name)

        if clayer.name.split("_")[0] in ["block{}".format(i) for i in range(10, 15)]:
            clayer.trainable = True


    model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(lr=learning_rate[sub_model_order]),
                    metrics=[top_k_categorical_accuracy, 'accuracy'])


    model.fit_generator(generator=train_generator,
                        steps_per_epoch=math.ceil(2000000 / batch_size),
                        verbose=1,
                        callbacks=callbacks[sub_model_order],
                        validation_data=validation_generator,
                        initial_epoch=3,
                        epochs = num_epochs[sub_model_order],
                        use_multiprocessing=True,
                        max_queue_size=10,
                        workers = 8,
                        validation_steps=math.ceil(10000 / batch_size))


    model.save(sub_model_save_paths[sub_model_order])
    #save_model(model, sub_model_save_paths[sub_model_order])

## train WHOLE model

In [ ]:
sub_model_order = 2  # whole model (base 0)

if had_been_run[sub_model_order]:
    print("\n We FOUND",sub_model_save_paths[sub_model_order],".\n\t We will skip this model.\n\n")

else:

    if had_been_run[sub_model_order - 1]:
        model = load_model(sub_model_save_paths[sub_model_order - 1])  # returns a compiled model identical to the previous one

    print("\n Starting optimization of model:",sub_model_names[sub_model_order])
    print('\n\n\n Debugging whole model model: ')
    for lay in model.layers:
        print(lay)
    print('\n\n\n')


    for clayer in model.layers[num_gpus_to_use + 1].layers:
        clayer.trainable = True

    # Note you need to recompile the whole thing. Otherwise you are not traing first layers
    model.compile(loss='categorical_crossentropy',
                    optimizer=Adam(lr=learning_rate[sub_model_order]),
                    metrics=[top_k_categorical_accuracy, 'accuracy'])


    init_epochs = 8  # We pretrained the model already

    # Keep training for as long as you like.
    for i in range(100):
        # gradually decrease the learning rate
        K.set_value(model.optimizer.lr, 0.95 * K.get_value(model.optimizer.lr))
        start_epoch = (i * 2)
        epochs = ((i + 1) * 2)
        model.fit_generator(generator=train_generator,
                            steps_per_epoch=math.ceil(2000000 / batch_size),
                            verbose=1,
                            callbacks=callbacks[sub_model_order],
                            validation_data=validation_generator,
                            initial_epoch=start_epoch + init_epochs,
                            epochs=epochs + init_epochs,
                            use_multiprocessing=True,
                            max_queue_size=10,
                            workers = 8,
                            validation_steps=math.ceil(10000 / batch_size))


        model.save(model, models_folder + "mod_and_outp_" + model_name + "_" + subm + "_ep_" + str(i) + ".hdf5")